# Bring your own tools to the enterprise gateway
In this notebook, you'll augment the existing AgentCore Gateway with your own tools of choice. This is your chance to experiment and publish reusable tools that could be made available to others via the gateway. 
- Lambda function targets
- OpenAPI schema targets
- Smithy model targets
- MCP servers targets
- Built-in templates from integration providers as targets

You can find more information for each of these target options  here: https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway-supported-targets.html 

![List](images/mcp-server-list-tools.png)

## 1. Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude Sonnet 4 model on Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
%pip install -U boto3 strands-agents strands-agents-tools defusedxml httpx bedrock_agentcore_starter_toolkit

## 2. Lambda targets: 

In the previous lab, we used the database tools from Biomni as a lambda function target for the AgentCore Gateway. You could explore reusing other tool categories from Biomni as available [here](https://github.com/snap-stanford/Biomni/tree/main/biomni/tool) . 

For example, we demonstrate literature tools from Biomni as a second target to the gateway  in the [AWS Life sciences sample agents repository](https://github.com/aws-samples/amazon-bedrock-agents-healthcare-lifesciences/tree/main/agents_catalog/28-Research-agent-biomni-gateway-tools/prerequisite/lambda-literature). 

You can add the tools to your existing Gateway as another lambda target with the help of this [notebook demonstration](https://github.com/awslabs/amazon-bedrock-agentcore-samples/blob/main/01-tutorials/02-AgentCore-gateway/01-transform-lambda-into-mcp-tools/01-gateway-target-lambda-oauth.ipynb)


## 3. MCP server as a target

Another option is to bring a MCP server as a target. AgentCore Gateway now supports onboarding pre-existing MCP server implementations as targets, in addition to Lambda and OpenAPI tools. 

Create the MCP server target for the gateway

In [ ]:
# Try with AWS Knowledge MCP Server
# https://awslabs.github.io/mcp/servers/aws-knowledge-mcp-server

import boto3
from utils import get_ssm_parameter

gatewayID = get_ssm_parameter("/deep-research-workshop/agentcore/gateway_id")
# Get gateway endpoint
gateway_endpoint = get_ssm_parameter("/deep-research-workshop/agentcore/gateway_url")
print(f"Gateway Endpoint - MCP URL: {gateway_endpoint}")
gateway_client = boto3.client("bedrock-agentcore-control")

# Create the Gateway Target for the AWS Knowledge MCP Server
create_gateway_target_response = gateway_client.create_gateway_target(
    name="aws-knowledge-server-target",
    gatewayIdentifier=gatewayID,
    targetConfiguration={
        "mcp": {"mcpServer": {"endpoint": "https://knowledge-mcp.global.api.aws"}}
    },
)
print(create_gateway_target_response)

In [ ]:
# Check that the Gateway target exists, and is READY
list_targets_response = gateway_client.list_gateway_targets(gatewayIdentifier=gatewayID)
print(list_targets_response)

### Check if the new MCP Server tools are now available through the Gateway

In [ ]:
from database_tools import (
    get_gateway_access_token,
    get_all_mcp_tools_from_mcp_client,
)
from utils import get_ssm_parameter
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp import MCPClient
import time

# Get gateway access token
jwt_token = get_gateway_access_token()
if not jwt_token:
    print("❌ Failed to get gateway access token")

# Get gateway endpoint
gateway_endpoint = get_ssm_parameter("/deep-research-workshop/agentcore/gateway_url")
print(f"Gateway Endpoint - MCP URL: {gateway_endpoint}")

# Create MCP client
client = MCPClient(
    lambda: streamablehttp_client(
        gateway_endpoint, headers={"Authorization": f"Bearer {jwt_token}"}
    )
)

with client:
    print("📋 Getting all available tools...")
    start_time = time.time()
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    list_time = time.time() - start_time
    print(f"✅ Found {len(all_tools)} total tools in {list_time:.2f}s\n")
    for tool in all_tools:
        print(tool.tool_name)

## Next steps : Augment your agent to access the new MCP server tools via the Gateway